In [1]:
%pwd

'/home/giakhang/dev/pose_sandbox/Hand_pose_estimation_3D/arm_and_hand'

In [2]:
import cv2
import numpy as np
from mmdeploy_runtime import Detector, PoseDetector

In [3]:
def visualize(frame, keypoints, filename, thr=0.5, resize=1280):
    skeleton = [
        [0, 5], [5, 7], [7, 92], 
        [92, 93], [93, 94], [94, 95], [95, 96],
        [92, 97], [97, 98], [98, 99], [99, 100],
        [101, 102], [102, 103], [103, 104],
        [105, 106], [106, 107], [107, 108],
        [92, 109], [109, 110], [110, 111], [111, 112],
        [93, 97], [97, 101], [101, 105], [105, 109]
    ]

    palette = [
        # Body colors
        (255, 128, 0), (255, 153, 51), (255, 178, 102), (230, 230, 0), 
        (255, 153, 255), (153, 204, 255), (255, 102, 255), (255, 51, 255),
        (102, 178, 255), (51, 153, 255), (255, 153, 153), (255, 102, 102), 
        (255, 51, 51), (153, 255, 153), (102, 255, 102), (51, 255, 51),
        (0, 255, 0), (0, 0, 255), (255, 0, 0),
        # Face colors
        (255, 255, 0), (255, 255, 102), (255, 255, 153), (255, 255, 204),
        # Right Hand colors (21 keypoints)
        (153, 0, 255), (178, 102, 255), (204, 153, 255), (230, 204, 255),
        (255, 204, 255), (204, 102, 255), (178, 51, 255), (153, 51, 255),
        (128, 51, 255), (102, 51, 255), (51, 51, 255), (102, 102, 255),
        (153, 153, 255), (204, 204, 255), (230, 230, 255), (255, 230, 255),
        (255, 153, 230), (255, 102, 230), (255, 51, 230), (255, 0, 230),
        # Left Hand colors (21 keypoints)
        (0, 153, 255), (51, 178, 255), (102, 204, 255), (153, 230, 255),
        (204, 255, 255), (51, 255, 204), (51, 255, 153), (51, 255, 102),
        (51, 255, 51), (102, 255, 102), (153, 255,153), (153, 255,204),
        (153, 255,255), (204, 255,255), (255, 255,204), (255, 255,153),
        (255, 255,102), (255, 255,51), (255, 255,0), (230, 230,0)
    ]

    link_color = [
        # Body links
        0, 0, 0, 0, 7, 7, 7, 9, 9, 9, 9, 9, 16, 16, 16, 16, 16, 16, 16,
        # Face links
        4, 4, 4, 4, 4, 4, 4, 4,
        # Right Hand links (21)
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        # Left Hand links (21)
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6
    ]

    point_color = [
        # Body points
        16, 16, 16, 16, 16, 9, 9, 9, 9, 9, 9, 0, 0, 0, 0, 0, 0,
        # Face points
        4, 4, 4, 4, 4, 4, 4, 4,
        # Right Hand points (21)
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        # Left Hand points (21)
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6
    ]

    scale = resize / max(frame.shape[0], frame.shape[1])

    scores = keypoints[..., 2]
    keypoints = (keypoints[..., :2] * scale).astype(int)

    img = cv2.resize(frame, (0, 0), fx=scale, fy=scale)
    for kpts, score in zip(keypoints, scores):
        show = [0] * len(kpts)
        for (u, v), color in zip(skeleton, link_color):
            if score[u] > thr and score[v] > thr:
                cv2.line(img, kpts[u], tuple(kpts[v]), palette[color], 1,
                         cv2.LINE_AA)
                show[u] = show[v] = 1
        for kpt, show, color in zip(kpts, show, point_color):
            if show:
                cv2.circle(img, kpt, 1, palette[color], 2, cv2.LINE_AA)
    cv2.imwrite(filename, img)

In [4]:
person_detector = Detector(model_path="/home/giakhang/dev/pose_sandbox/mmdeploy/rtmdet-trt/rtmdet_nano", 
    device_name="cuda", 
    device_id=0)

pose_detector = PoseDetector(model_path="/home/giakhang/dev/pose_sandbox/mmdeploy/rtmpose-trt/rtmpose-m/rtmpose-m_left_cam",
    device_name="cuda",
    device_id=0)

[2024-10-25 10:32:12.664] [mmdeploy] [info] [model.cpp:35] [DirectoryModel] Load model: "/home/giakhang/dev/pose_sandbox/mmdeploy/rtmdet-trt/rtmdet_nano"
[2024-10-25 10:32:12.915] [mmdeploy] [info] [model.cpp:35] [DirectoryModel] Load model: "/home/giakhang/dev/pose_sandbox/mmdeploy/rtmpose-trt/rtmpose-m/rtmpose-m_left_cam"


In [5]:
img = cv2.imread("/home/giakhang/dev/pose_sandbox/data/2024-10-19/2024-10-19-15:39/image/left/train/left_110.jpg")

In [6]:
bboxes, labels, _ = person_detector(img)

In [10]:
type(bboxes)

numpy.ndarray

In [13]:
bboxes.shape

(45, 5)

In [14]:
keep = np.logical_and(labels == 0, bboxes[..., 4] > 0.6)
bboxes = bboxes[keep, :4]

In [15]:
pose = pose_detector(img, bboxes)

In [17]:
pose.shape

(1, 133, 3)

In [18]:


pose

array([[[9.93585938e+02, 4.89839142e+02, 1.17729306e+00],
        [1.01841406e+03, 4.64973907e+02, 1.26178288e+00],
        [9.70531250e+02, 4.68526093e+02, 1.24245334e+00],
        [1.06275000e+03, 4.61421722e+02, 1.25962949e+00],
        [9.47476562e+02, 4.70302155e+02, 1.16806483e+00],
        [1.12836719e+03, 6.07060913e+02, 1.46490479e+00],
        [9.27968750e+02, 5.64434814e+02, 1.30768752e+00],
        [1.15851562e+03, 7.91773926e+02, 1.78613663e+00],
        [8.81859375e+02, 5.94628296e+02, 1.25838208e+00],
        [1.05033594e+03, 8.87682617e+02, 1.55806851e+00],
        [8.49937500e+02, 5.53778320e+02, 1.59663999e+00],
        [1.06452344e+03, 9.55173950e+02, 1.47464061e+00],
        [9.29742188e+02, 9.32084717e+02, 1.32210195e+00],
        [1.06097656e+03, 1.07417175e+03, 7.22713828e-01],
        [9.33289062e+02, 1.07417175e+03, 1.22915709e+00],
        [1.26669531e+03, 3.26439117e+02, 4.13701439e+00],
        [1.26669531e+03, 3.24663055e+02, 3.24966097e+00],
        [1.266

In [19]:
9.93585938e+02

993.585938

In [9]:
visualize(img, pose, "det_pose_output.jpg", 0.5, 1280)

In [10]:
!pwd

/home/giakhang
